In [ ]:
from lxml import html
import requests
import pandas as pd
import re

In [ ]:
# Pull out all the links and congressmembers names from webpage.
page = requests.get('https://www.opensecrets.org/members-of-congress/members-list?cong_no=117&cycle=2020')
webpage = html.fromstring(page.content)
list_of_hrefs = webpage.xpath("//a/@href")
list_of_names = webpage.xpath("//a/text()")
culled_href_list = []
cid = re.compile(r'(N\d+)')
for i in list_of_hrefs:
    if 'summary?cid=' in i:
        cid_found = cid.search(i)
        culled_href_list.append(cid_found.group(1))
list_of_names = list_of_names[list_of_names.index('Abraham, Ralph'):list_of_names.index('Zeldin, Lee')+1]
list_of_names = [str(i) for i in list_of_names]
list_of_names = [(j[j.index(','):] + ' ' + j[:j.index(',')]).replace(', ','') for j in list_of_names]
names_and_CIDs = pd.DataFrame(data = culled_href_list, 
                               index = list_of_names, 
                               columns = ['CID']).reset_index().rename(columns = {'index':'Name'})

In [ ]:
just_states = pd.read_csv("Write_Data_Here/reps_plus_abbreviations.csv", usecols = ['State'])
just_states = list(just_states['State'])
just_states.extend(['Puerto Rico', 'Virgin Islands', 'Guam'])
just_parties = ['Republican', 'Democrat', 'Third Party', 'Independent']
other_info_df = pd.DataFrame(columns = ['Mashed Up','State','Party','Senator or Representative'])
list_of_other_information = webpage.xpath("//i/text()")
other_info_df['Mashed Up'] = [str(i) for i in list_of_other_information]
for i in other_info_df['Mashed Up']:
    for j in just_states:
        if j in i:
            other_info_df['State'].loc[other_info_df['Mashed Up'].isin([i])] = j
    for k in just_parties:
        if k in i:
            other_info_df['Party'].loc[other_info_df['Mashed Up'].isin([i])] = k
    if 'District' in i:
        other_info_df['Senator or Representative'].loc[other_info_df['Mashed Up'].isin([i])] = 'Representative'
    else:
        other_info_df['Senator or Representative'].loc[other_info_df['Mashed Up'].isin([i])] = 'Senator'
names_and_CIDs = pd.concat(objs = [names_and_CIDs, other_info_df.drop(columns = 'Mashed Up')], axis = 1)

In [ ]:
def use_URL_to_create_df(URL):
    headers = {'User-Agent': 'Mozilla/5.0'}
    page = requests.get(URL, headers = headers)
    webpage = html.fromstring(page.content)

    def take_out_leading_characters(given_string):
        return given_string.text_content().replace('\n\t\t\t\t\t\t\t\t','')

    def pull_out_table_data(webpage):
        for table in webpage.xpath('.//table[@class="DataTable"]'):
            header = [text(table_header) for table_header in table.xpath('//th')]
            cells_interlinked = (
                [[text(table_cell) for table_cell in table_row.xpath('td')] for table_row in table.xpath('//tr')]
            )
            cells_interlinked = [row for row in cells_interlinked if len(row) == len(header)] 
            contributor_df = pd.DataFrame(cells_interlinked, columns = header)

            for i in contributor_df.columns:
                if i != 'Rank' and i != 'Contributor':
                    contributor_df[i] = (
                        contributor_df[i].\
                        str.replace('$','', regex = False).\
                        str.replace(',','', regex = False).\
                        astype('int64')
                    )

        return contributor_df
    
    contributor_df = pull_out_table_data(webpage)
    
    cid = re.compile(r'(N\d+)')
    cid_found = cid.search(URL)
    contributor_df['CID'] = cid_found.group(1)
    
    return contributor_df